In [1]:
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import AdamW


In [2]:
batch_size = 32
block_size = 8
seed = 1337
g = torch.manual_seed(seed)

In [3]:
with open('./input.txt') as f:
    text = f.read()

print(f"{len(text) = } characters",
       end='\n-------------Data-------------\n')

print(text[:100])

len(text) = 1115394 characters
-------------Data-------------
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


### Build our char level vocabulary

In [4]:
chars = sorted(list(set(text)))

stoi = {ch:idx for idx, ch in enumerate(chars)}
itos = {idx:ch for idx, ch in enumerate(chars)}

vocab_size = len(chars)
print(vocab_size)

65


In [5]:
encode = lambda text: [stoi[c] for c in text]
decode = lambda tokens: ''.join([itos[c] for c in tokens])

raw = 'i love this game.'
tokens = encode(raw)
print(tokens)
print(decode(tokens))

[47, 1, 50, 53, 60, 43, 1, 58, 46, 47, 57, 1, 45, 39, 51, 43, 8]
i love this game.


### Encode entire data

In [6]:
data = torch.tensor(encode(text)).long()
print(data.shape, data.type())
print(data[:100])

torch.Size([1115394]) torch.LongTensor
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


### Data split

In [7]:
train_data, val_data = train_test_split(data, shuffle=False, test_size=0.1)
print(train_data[:block_size])

tensor([18, 47, 56, 57, 58,  1, 15, 47])


In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for step in range(block_size):
    context = x[:step+1] ## step+1 as range indexing is exclusive
    target = y[step]
    print(f"Step {step+1} context is {context}, target is {target}")

Step 1 context is tensor([18]), target is 47
Step 2 context is tensor([18, 47]), target is 56
Step 3 context is tensor([18, 47, 56]), target is 57
Step 4 context is tensor([18, 47, 56, 57]), target is 58
Step 5 context is tensor([18, 47, 56, 57, 58]), target is 1
Step 6 context is tensor([18, 47, 56, 57, 58,  1]), target is 15
Step 7 context is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
Step 8 context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [9]:
def get_batch(split_type):
    data = train_data if split_type=='train' else val_data
    ix = torch.randint(low=0, high=len(data)-block_size, size=(batch_size,), generator=g)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [10]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb[0])
print('targets:')
print(yb.shape)
print(yb[0])


inputs:
torch.Size([32, 8])
tensor([24, 43, 58,  5, 57,  1, 46, 43])
targets:
torch.Size([32, 8])
tensor([43, 58,  5, 57,  1, 46, 43, 39])


In [11]:
class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.table = nn.Embedding(vocab_size, vocab_size)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, idx, targets=None):

        logits = self.table(idx)

        if targets is None:
            loss = None
        else:
            batch, seq_len, embds = logits.shape
            logits = logits.view(batch*seq_len, embds)
            targets = targets.view(batch*seq_len)
            loss = self.loss(logits, targets)
        
        return logits, loss
    

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
## In this code we still preserve history tokens, however we don't use them.


In [ ]:
m = BigramLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

input = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(input, max_new_tokens=100)[0].tolist()))

optimizer = AdamW(m.parameters(), lr=1e-3)

torch.Size([256, 65])
tensor(4.6922, grad_fn=<NllLossBackward0>)

sV
vL
ja,FsLY,wxEuS'pao3jOssyBA$zFqYTkeMk x-gQ.FzLg!iKI.egzDnyA TsTbvdgX!KpGIeJyjv,SrFF&SDt!:hwWSl.W


In [13]:
batch_size = 32
for steps in range(5000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

with torch.no_grad():
    input = torch.zeros((1, 1), dtype=torch.long)
    print(decode(m.generate(input, max_new_tokens=100)[0].tolist()))

2.5431320667266846

INTI heave anl!
ALTus! bsee pout yo f'd, t fed inks cerVFin, I hasoweppoof pre, hairal.
Tir W:KIELg 
